In [2]:
!pip install clouddrift numpy s3fs xarray

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.7/831.7 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.2/636.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 74.5 MB/s eta 0:00:00
  Created wheel for asciitree: filename=asciitree-0.3.3-py3-none-any.whl size=5032 sha256=1dec23f0beea01af5eb10a0d304e4d32ef46fc77d6227bd095bb6e85f233f4c9
  Stored in directory: /home/onyxia/.cache/pip/wheels/a5/d7/98/f56ae733748cd0fa577172bda0e73e0b1f1793c98e09b9e458
Successfully built asciitree


In [3]:
import clouddrift as cd
import numpy as np
import s3fs
import xarray as xr

## Access EDITO s3

In [9]:
s3 = s3fs.S3FileSystem(
  anon=True,
  endpoint_url="https://minio.lab.dive.edito.eu"
)

In [10]:
s3.ls("/oidc-bertrava/data")

['oidc-bertrava/data/4dvarnet.zarr',
 'oidc-bertrava/data/duacs.zarr',
 'oidc-bertrava/data/gdp6h.zarr',
 'oidc-bertrava/data/miost.zarr',
 'oidc-bertrava/data/vardyn.zarr']

## Sea surface currents

In [12]:
vardyn_store = s3fs.S3Map(root=f"s3://oidc-bertrava/data/vardyn.zarr", s3=s3)

In [13]:
ds_vardyn = xr.open_zarr(vardyn_store)
ds_vardyn

<xarray.Dataset> Size: 4GB
Dimensions:             (time: 465, latitude: 276, longitude: 600)
Coordinates:
  * latitude            (latitude) float32 1kB 25.0 25.09 25.18 ... 49.91 50.0
  * longitude           (longitude) float64 5kB 280.0 280.1 ... 349.8 349.9
  * time                (time) datetime64[ns] 4kB 2023-07-27 ... 2023-11-20
Data variables:
    adt                 (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>
    relative_vorticity  (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>
    sla                 (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>
    ugos                (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>
    ugosa               (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>
    vgos                (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>
    vgosa               (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>

Longitudes range from 0° to 360°: need to transform them to work in [-180°, 180°].

In [17]:
ds_vardyn["longitude"] = ds_vardyn.longitude - 180
ds_vardyn

<xarray.Dataset> Size: 4GB
Dimensions:             (time: 465, latitude: 276, longitude: 600)
Coordinates:
  * latitude            (latitude) float32 1kB 25.0 25.09 25.18 ... 49.91 50.0
  * longitude           (longitude) float64 5kB 100.0 100.1 ... 169.8 169.9
  * time                (time) datetime64[ns] 4kB 2023-07-27 ... 2023-11-20
Data variables:
    adt                 (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>
    relative_vorticity  (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>
    sla                 (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>
    ugos                (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>
    ugosa               (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>
    vgos                (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>
    vgosa               (time, latitude, longitude) float64 616MB dask.array<chunksize=(1, 276, 600), meta=np.ndarray>

### GDP Drifters

In [18]:
ds_gdp1h = cd.datasets.gdp1h()
ds_gdp1h

<xarray.Dataset> Size: 16GB
Dimensions:                (traj: 19396, obs: 197214787)
Coordinates:
    id                     (traj) int64 155kB ...
    time                   (obs) datetime64[ns] 2GB ...
Dimensions without coordinates: traj, obs
Data variables: (12/59)
    BuoyTypeManufacturer   (traj) |S20 388kB ...
    BuoyTypeSensorArray    (traj) |S20 388kB ...
    CurrentProgram         (traj) float32 78kB ...
    DeployingCountry       (traj) |S20 388kB ...
    DeployingShip          (traj) |S20 388kB ...
    DeploymentComments     (traj) |S20 388kB ...
    ...                     ...
    start_lat              (traj) float32 78kB ...
    start_lon              (traj) float32 78kB ...
    typebuoy               (traj) |S10 194kB ...
    typedeath              (traj) int8 19kB ...
    ve                     (obs) float32 789MB ...
    vn                     (obs) float32 789MB ...
Attributes: (12/16)
    Conventions:       CF-1.6
    acknowledgement:   Elipot, Shane; Sykulski, Adam; Lumpkin, Rick; Centurio...
    contributor_name:  NOAA Global Drifter Program
    contributor_role:  Data Acquisition Center
    date_created:      2023-09-08T17:05:12.130123
    doi:               10.25921/x46c-3620
    ...                ...
    processing_level:  Level 2 QC by GDP drifter DAC
    publisher_email:   aoml.dftr@noaa.gov
    publisher_name:    GDP Drifter DAC
    publisher_url:     https://www.aoml.noaa.gov/phod/gdp
    summary:           Global Drifter Program hourly data
    title:             Global Drifter Program hourly drifting buoy collection

In [19]:
ds_gdp1h.time.max()

<xarray.DataArray 'time' ()> Size: 8B
array('2022-10-31T00:00:00.000000000', dtype='datetime64[ns]')

Clouddrift server can crash, use our own zarr

In [20]:
gdp6h_store = s3fs.S3Map(root=f"s3://oidc-bertrava/data/gdp6h.zarr", s3=s3)

In [21]:
ds_gdp6h = xr.open_zarr(gdp6h_store)
ds_gdp6h

<xarray.Dataset> Size: 2GB
Dimensions:                (traj: 27647, obs: 46535470)
Coordinates:
    id                     (traj) int64 221kB dask.array<chunksize=(27647,), meta=np.ndarray>
    time                   (obs) datetime64[ns] 372MB dask.array<chunksize=(181780,), meta=np.ndarray>
Dimensions without coordinates: traj, obs
Data variables: (12/49)
    BuoyTypeManufacturer   (traj) |S20 553kB dask.array<chunksize=(13824,), meta=np.ndarray>
    BuoyTypeSensorArray    (traj) |S20 553kB dask.array<chunksize=(13824,), meta=np.ndarray>
    CurrentProgram         (traj) float64 221kB dask.array<chunksize=(27647,), meta=np.ndarray>
    DeployingCountry       (traj) |S20 553kB dask.array<chunksize=(13824,), meta=np.ndarray>
    DeployingShip          (traj) |S20 553kB dask.array<chunksize=(13824,), meta=np.ndarray>
    DeploymentComments     (traj) |S20 553kB dask.array<chunksize=(13824,), meta=np.ndarray>
    ...                     ...
    start_lon              (traj) float32 111kB dask.array<chunksize=(27647,), meta=np.ndarray>
    temp                   (obs) float32 186MB dask.array<chunksize=(363559,), meta=np.ndarray>
    typebuoy               (traj) |S10 276kB dask.array<chunksize=(13824,), meta=np.ndarray>
    typedeath              (traj) int8 28kB dask.array<chunksize=(27647,), meta=np.ndarray>
    ve                     (obs) float32 186MB dask.array<chunksize=(363559,), meta=np.ndarray>
    vn                     (obs) float32 186MB dask.array<chunksize=(363559,), meta=np.ndarray>
Attributes: (12/18)
    Conventions:          CF-1.6
    acknowledgement:      Lumpkin, Rick; Centurioni, Luca (2019). NOAA Global...
    contributor_name:     NOAA Global Drifter Program
    contributor_role:     Data Acquisition Center
    date_created:         2024-04-04T13:44:01.176967
    doi:                  10.25921/7ntx-z961
    ...                   ...
    publisher_name:       GDP Drifter DAC
    publisher_url:        https://www.aoml.noaa.gov/phod/gdp
    summary:              Global Drifter Program six-hourly data
    time_coverage_end:    2023-10-18:18:00:00Z
    time_coverage_start:  1979-02-15:00:00:00Z
    title:                Global Drifter Program drifting buoy collection

In [23]:
ds_gdp6h.time.values.max()

np.datetime64('2023-10-18T18:00:00.000000000')

MIOST, VarDyn, 4dVarNet datasets from Aviso+ start at 2023-07-27: we will have to use the GDP6H dataset.

In [ ]:
ds_drifters = cd.ragged.subset(
    ds_gdp6h, 
    {
        "time": (np.datetime64("2023-07-27"), np.datetime64("2023-11-20")),
        "lat": (25, 50),
        "lon": (100, 169.9)
    }, 
    row_dim_name="traj"
)
ds_drifters